<a href="https://colab.research.google.com/github/Shashidhar-Gowda/ChatBot/blob/mlds/LR%2Bsummaryv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install langchain langchain-community scikit-learn pandas groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.50
    Uninstalling langchain-core-0.3.50:
      Successfully uninstalled langchain-core-0.3.50
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.7
    Uninstalling langchain-text-splitters-0.3.7:
      Successfully uninstalled langchain-text-splitters-0.3.7
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.22
    Uninstalling langchain-0.3.22:
      Successfully uninstalled langchain-0.3.22


In [2]:
pip install -qU "langchain[groq]"

In [3]:
import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("deepseek-r1-distill-llama-70b", model_provider="groq")

Enter API key for Groq: ··········


In [4]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

def perform_regression(df: pd.DataFrame):
    df = df.dropna()
    num_cols = df.select_dtypes(include='number').columns.tolist()
    if len(num_cols) < 2:
        return "Not enough numerical columns for regression."

    X = df[num_cols[:-1]]
    y = df[num_cols[-1]]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    model = LinearRegression()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    results = {
        "coefficients": dict(zip(X.columns, model.coef_)),
        "intercept": model.intercept_,
        "r2_score": r2_score(y_test, y_pred),
        "mse": mean_squared_error(y_test, y_pred)
    }

    return results


In [6]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

template = """
You are a data analyst. Based on the following linear regression output, create a professional summary:

Regression Output:
{regression_output}

Include:
- Key predictors and their coefficients
- R-squared and MSE values
- Interpretation of the model fit
- Any interesting insights
"""

prompt = PromptTemplate(
    input_variables=["regression_output"],
    template=template,
)

chain = LLMChain(llm=model, prompt=prompt)


<ipython-input-6-338f0e24fd25>:22: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=model, prompt=prompt)


In [7]:
# Step 1: Load dataset
df = pd.read_csv("/content/assignment_data.csv")

# Step 2: Run regression
results = perform_regression(df)

# Step 3: Generate summary
if isinstance(results, dict):
    summary = chain.run(regression_output=str(results))
    print(summary)
else:
    print(results)


<ipython-input-7-2b631a55b9fb>:9: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  summary = chain.run(regression_output=str(results))


<think>
Alright, I've got this query from a user who is a data analyst. They provided a linear regression output and want a professional summary based on that. Let me start by understanding what they need.

First, I need to look at the regression output. It includes coefficients for various predictors, an intercept, R-squared, and MSE. My task is to extract key predictors and their coefficients, include R-squared and MSE, interpret the model fit, and find any interesting insights.

I should start by identifying the key predictors. Looking at the coefficients, some are positive and some are negative. The user probably wants to know which variables have the most impact, both positive and negative. So I'll list the top few from each category.

Next, I need to explain the R-squared and MSE. R-squared is about 0.52, which means the model explains over half the variance. That's moderate, not too strong but decent. MSE is 37, which I guess is the units of the dependent variable. I should ment